In [1]:
import json
import pandas as pd
from tqdm import tqdm
dataset = 'Yelp'
stage='item'

In [3]:
with open(f'input/caption/{dataset}_caption.json', 'r') as file:
    name = json.load(file)

In [ ]:
len(name)

In [ ]:
prompt = """Based on the store name and your general knowledge, infer the key characteristics of the store. Include details such as its type (e.g., restaurant, mall, etc.), potential offerings or services, and target audience. Ensure the response is concise and informative.
Example:
Store Name: Bar-B-Cutie
Response:
Type: Restaurant (Barbecue-focused)
Offerings: Likely specializes in barbecue dishes such as smoked meats, ribs, and sides. May also serve casual American comfort food.
Target Audience: Barbecue enthusiasts, families, or individuals seeking a casual dining experience.
Now, analyze the following store name:
Store Name: {}
Response:
"""

data_list = []
file_count = 1

for id, caption in tqdm(name.items()):
    data_entry = {
        "custom_id": str(id),
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "messages": [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt.format(caption)}
            ]
        }
    }
    data_list.append(data_entry)

    # 每45000条数据写入一个新文件
    if len(data_list) >= 30000:
        with open(f'{dataset}_{stage}request_{file_count}.jsonl', 'w', encoding='utf-8') as f:
            for entry in data_list:
                json.dump(entry, f, ensure_ascii=False)
                f.write('\n')
        file_count += 1
        data_list = []

# 写入剩余的数据
if data_list:
    with open(f'{dataset}_{stage}request_{file_count}.jsonl', 'w', encoding='utf-8') as f:
        for entry in data_list:
            json.dump(entry, f, ensure_ascii=False)
            f.write('\n')